### 任务一. 人脸检测和识别流程

本案例任务的主要目标是运行一个 完整的人脸检测和识别的流程。

#### 0. 环境准备

安装本案例所需的依赖包

In [1]:
!pip install facenet_pytorch==2.5.2

Looking in indexes: https://pypi.virtaicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.4 MB/s eta 0:00:0000:01


In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


根据当前运行环境是否有GPU可用，设置device的值

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('在该设备上运行: {}'.format(device))

在该设备上运行: cuda:0


##### 准备模型文件

自己动手将所挂载的facenet_data数据集的目录下两个模型文件（`20180402-114759-vggface2.pt`、`20180408-102900-casia-webface.pt`）复制到默认模型读取路径`“~/.cache/torch/checkpoints”`，请打开终端动手操作。

#### 1. 定义MTCNN模块

为了说明，默认参数已显示，但不是必需的。请注意，由于MTCNN是一组神经网络和其他代码，因此必须以以下方式传递设备，以便在需要内部复制对象时启用。

关键字参数：
|      image_size {int} -- 输出图像大小，以像素为单位。图像将是正方形。（默认值：{160}）


|      margin {int} -- 给定最终图像中边界框添加的边缘，以像素为单位          （默认值：{0}）

|      min_face_size {int} -- 要搜索的最小人脸尺寸。（默认值：{20}）

|      thresholds {list} -- MTCNN人脸检测阈值（默认值：{[0.6, 0.7, 0.7]}）

|      factor {float} -- 用于创建人脸尺寸的缩放金字塔的因子。（默认值：{0.709}）

|      post_process {bool} -- 在返回之前是否对图像张量进行后处理。
|          （默认值：{True}）

|      select_largest {bool} -- 如果为True，如果检测到多张人脸，将返回最大的人脸。
|          如果为False，将返回检测概率最高的人脸。
|          （默认值：{True}）

|      selection_method {string} -- 用于选择的启发式方法。默认为None。如果指定，将覆盖select_largest：
|                  "probability": 选择最高概率
|                  "largest": 选择最大的框
|                  "largest_over_threshold": 选择超过某个概率的最大框
|                  "center_weighted_size": 框大小减去从图像中心加权的平方偏移
|              （默认值：{None}）

|      keep_all {bool} -- 如果为True，将返回所有检测到的人脸，顺序由select_largest参数决定。如果指定了save_path，则第一个人脸保存到该路径，其余人脸保存到<save_path>1, <save_path>2等。
|          （默认值：{False}）

|      device {torch.device} -- 运行神经网络传递的设备。在运行前向传递之前，将图像张量和模型复制到此设备。（默认值：{None}）


查看`help(MTCNN)`获取更多细节。

In [4]:
help(MTCNN)

Help on class MTCNN in module facenet_pytorch.models.mtcnn:

class MTCNN(torch.nn.modules.module.Module)
 |  MTCNN(image_size=160, margin=0, min_face_size=20, thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, select_largest=True, selection_method=None, keep_all=False, device=None)
 |  
 |  MTCNN face detection module.
 |  
 |  This class loads pretrained P-, R-, and O-nets and returns images cropped to include the face
 |  only, given raw input images of one of the following types:
 |      - PIL image or list of PIL images
 |      - numpy.ndarray (uint8) representing either a single image (3D) or a batch of images (4D).
 |  Cropped faces can optionally be saved to file
 |  also.
 |  
 |  Keyword Arguments:
 |      image_size {int} -- Output image size in pixels. The image will be square. (default: {160})
 |      margin {int} -- Margin to add to bounding box, in terms of pixels in the final image. 
 |          Note that the application of the margin differs slightly from the 

<font color=red size=3>动手练习1</font> <br>

在`<1>`处创建MTCNN的一个实例，设置运行设备为上一步中检测出的设备，其它参数可保持默认值；


In [5]:
mtcnn = MTCNN(device=device)

### 2. 定义Inception Resnet V1模块

设置classify=True以使用预训练分类器。对于本示例，我们将使用该模型输出嵌入/卷积特征。请注意，在推理过程中，将模型设置为`eval`模式非常重要。

Inception Resnet V1模型，可以选择性地加载预训练的权重。模型参数可以根据在VGGFace2或CASIA-Webface数据集上的预训练进行加载。如果请求，预训练的状态字典在模型实例化时会自动下载，并缓存在torch的缓存中。后续的实例化会使用缓存而不是重新下载。

两个预训练模型都是在160x160像素的图像上训练的，因此如果应用于调整到这个尺寸的图像，它们将表现最佳。为了获得最佳结果，图像还应该使用MTCNN裁剪到的面部。

默认情况下，上述模型将返回图像的512维嵌入。要启用分类而不是嵌入，可以在模型构造函数中传递classify=True，或者之后使用model.classify = True设置对象属性。对于VGGFace2，预训练模型将输出长度为8631的logit向量，对于CASIA-Webface，将输出长度为10575的logit向量。

关键字参数：
- pretrained {str} -- 可选的预训练数据集。可以是 'vggface2' 或 'casia-webface'。（默认值：{None}）
- classify {bool} -- 模型是否应该输出分类概率或特征嵌入。（默认值：{False}）
- num_classes {int} -- 输出类别的数量。如果设置了 'pretrained' 并且 num_classes 不等于预训练模型使用的数量，最后的线性层将被随机初始化。（默认值：{None}）
- dropout_prob {float} -- Dropout概率。（默认值：{0.6}）

查看`help(InceptionResnetV1)`获取更多细节。

In [6]:
help(InceptionResnetV1)

Help on class InceptionResnetV1 in module facenet_pytorch.models.inception_resnet_v1:

class InceptionResnetV1(torch.nn.modules.module.Module)
 |  InceptionResnetV1(pretrained=None, classify=False, num_classes=None, dropout_prob=0.6, device=None)
 |  
 |  Inception Resnet V1 model with optional loading of pretrained weights.
 |  
 |  Model parameters can be loaded based on pretraining on the VGGFace2 or CASIA-Webface
 |  datasets. Pretrained state_dicts are automatically downloaded on model instantiation if
 |  requested and cached in the torch cache. Subsequent instantiations use the cache rather than
 |  redownloading.
 |  
 |  Keyword Arguments:
 |      pretrained {str} -- Optional pretraining dataset. Either 'vggface2' or 'casia-webface'.
 |          (default: {None})
 |      classify {bool} -- Whether the model should output classification probabilities or feature
 |          embeddings. (default: {False})
 |      num_classes {int} -- Number of output classes. If 'pretrained' is s

<font color=red size=3>动手练习2</font> <br>

在`<1>`处创建InceptionResnetV1的一个实例，加载"vggface2"预训练模型，并且设置为"eval"模式，最后将模型移动到指定的设备上运行。


In [7]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

### 3. 定义数据集和数据加载器

<font color=red size=3>动手练习3</font> <br>

将数据集中"test_images"文件夹复制到 `/gemini/code` 目录下。

在`<1>`处使用datasets.ImageFolder()加载图像数据集。这个类会读取目录，并假设每个子目录对应一个类别，目录名称是类别的名称。

在`<2>`处使用DataLoader类创建一个数据加载器，它将用于在训练或评估过程中批量加载数据。dataset参数是上面创建的ImageFolder实例。

In [8]:
#collate_fn被定义为返回批次中的第一个数据项。
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('data/test_images')

# 交换了类别名称和索引的关系，创建了一个从索引到类别名称的映射。
# 这在某些情况下很有用，比如在训练过程中需要根据索引快速查找类别名称。
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}

# 
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

### 4. 执行MTCNN人脸检测

迭代DataLoader对象并检测每个人脸及其关联的检测概率。如果检测到脸部，`MTCNN`的前向方法将返回裁剪到检测到的脸部的图像。


<font color=red size=3>动手练习4</font> <br>

在<1>处，使用mtcnn实例输入图像，返回检测到的人脸图像及其检测概率。

在<2>处，将索引y对应的类别名称添加入names中存储。

In [9]:
aligned = []  # 用于存储经过对齐处理的人脸图像。
names = []   # 用于存储与检测到的人脸对应的类别名称。
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('检测到的人脸及其概率: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

检测到的人脸及其概率: 0.999983
检测到的人脸及其概率: 0.999934
检测到的人脸及其概率: 0.999934
检测到的人脸及其概率: 0.999734
检测到的人脸及其概率: 0.999876
检测到的人脸及其概率: 0.999992


### 5. 计算图像嵌入

MTCNN将返回所有面部图像的相同大小，从而可以使用Resnet识别模块轻松进行批处理。在这里，由于我们只有一些图像，因此我们构建一个单个批次并对其执行推理。

<font color=red size=3>动手练习5</font> <br>

在<1>处，使用torch.stack()将之前收集到的aligned列表中的人脸图像堆叠成一个四维张量，并使用to(device) 将堆叠后的张量移动到之前定义的设备（device）上。

在<2>处，使用之前创建的resnet模型，将堆叠好的人脸张量aligned作为输入，进行前向传播以提取特征，并使用detach() 方法从当前计算图中分离出得到的张量，cpu() 方法将张量从当前设备（GPU）移动到CPU内存中（因为后续处理或存储在CPU上进行）。

In [10]:
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).detach().cpu()

#### 6. 计算各类别相互之间的距离

<font color=red size=3>动手练习6</font> <br>

在<1>处，计算e1和e2之间的欧氏距离。可以通过计算两个特征向量的差的范数（通常是L2范数）来实现的，并将范数张量转换成一个Python标量数值。

In [11]:
dists = [[(e2 - e1).norm().item() for e2 in embeddings] for e1 in embeddings]
pd.DataFrame(dists, columns=names, index=names)

,angelina_jolie,bradley_cooper,bradley_cooper,kate_siegel,paul_rudd,shea_whigham
angelina_jolie,0.000000,1.447517,1.447517,0.887726,1.434387,1.399102
bradley_cooper,1.447517,0.000000,0.000000,1.313755,1.012059,1.038734
bradley_cooper,1.447517,0.000000,0.000000,1.313755,1.012059,1.038734
kate_siegel,0.887726,1.313755,1.313755,0.000000,1.389084,1.379691
paul_rudd,1.434387,1.012059,1.012059,1.389084,0.000000,1.104222
shea_whigham,1.399102,1.038734,1.038734,1.379691,1.104222,0.000000
